In [115]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [116]:
imdb_data = pd.read_csv('./imdb_labelled.txt', sep='\t', names=['review', 'label'],
                        converters={'review': str.strip, 'label': int})
imdb_data.head()

,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [117]:
print(len(imdb_data[(imdb_data['label'] == 0)]))
print(len(imdb_data[(imdb_data['label'] == 1)]))

#tips[(tips['time'] == 'Dinner') & (tips['tip'] > 5.00)]

362
386


In [147]:
train_inp, test_inp, train_out, test_out =  train_test_split(imdb_data.review, imdb_data.label, test_size=0.2)
#, stratify=imdb_data.label, random_state=20)
print(len(train_inp))
print(len(test_inp))

598
150


In [148]:
from sklearn.naive_bayes import BernoulliNB as NaiveBayes
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
classifier = NaiveBayes()

In [149]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('extractor', vectorizer),
    ('classifier', classifier)
])

pipe.fit(train_inp, train_out)

Pipeline(memory=None,
     steps=[('extractor', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...lary=None)), ('classifier', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))])

In [150]:
#train_inp_vectorized = vectorizer.fit_transform(train_inp, train_out)
#classifier.fit(train_inp_vectorized, train_out)

In [151]:
print(classification_report(pipe.predict(test_inp), test_out))
print(accuracy_score(pipe.predict(test_inp), test_out))


             precision    recall  f1-score   support

          0       0.37      0.94      0.53        35
          1       0.97      0.50      0.66       115

avg / total       0.83      0.61      0.63       150

0.606666666667


In [152]:
print_top10(vectorizer, classifier, 'Positive Review')

Positive Review: time just acting like characters really great good movie film


In [153]:
from sklearn.metrics import confusion_matrix

y_pred = pipe.predict(test_inp)
confusion_matrix(test_out,y_pred)

array([[33, 57],
       [ 2, 58]], dtype=int64)

In [154]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,imdb_data.review, imdb_data.label, cv=5, scoring='recall')

array([ 0.73076923,  0.61038961,  0.87012987,  0.81818182,  0.92207792])